In [1]:

#improt basic packages
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw.MolDrawing import MolDrawing, DrawingOptions
import pandas as pd
import numpy as np
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from IPython.core.interactiveshell import InteractiveShell
#from pandas_profiling import ProfileReport
#import umap

InteractiveShell.ast_node_interactivity = "all"


In [ ]:
DrawingOptions.bondLineWidth=1.8
mol = Chem.MolFromSmiles('Brc1c(N)c(CNC2CCC(O)CC2)cc(Br)c1')
mol2=Chem.MolFromSmiles('C1CC(CCC1NCC2=C(C(=CC(=C2)Br)Br)N)O')
mol3=Chem.MolFromSmiles('[As](=O)(O)(O)c1ccc(NC(=O)N)cc1')
Descriptors.MolLogP(mol)
Descriptors.MolLogP(mol2)
mol
mol2
mol3
rdkit.Chem.rdMolDescriptors.BCUT2D(mol3)

In [ ]:
df = pd.read_csv('/home/wzabsd/Desktop/NAIST-PBL1/datasets/3CL_enzymatic_activity-clean.tsv', sep='\t')
#df.head()
profile = ProfileReport(df, title="Pandas Profiling Report")


In [ ]:

smiles_l=df['washed_SMILES'].tolist()
smiles_l[smiles_l.index('[i+]1c2c(c3c1cccc3)cccc2')]='C1=CC=C2C(=C1)C3=CC=CC=C3[I+]2'
#smiles_l.head()
mols_l=[Chem.MolFromSmiles(smile) for smile in smiles_l]
#Chem.MolToMolFile(mols_l, '/home/wzabsd/Desktop/NAIST-PBL1/output.mol')

In [ ]:
des_l=[des_name[0] for des_name in Descriptors._descList]
desc_calc = MoleculeDescriptors.MolecularDescriptorCalculator(des_l)
descriptors = pd.DataFrame([desc_calc.CalcDescriptors(mol) for mol in mols_l])
descriptors.columns = des_l
descriptors.index = smiles_l

In [ ]:
features=descriptors


In [ ]:
descriptors.head()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
df_traning = pd.read_csv('/home/wzabsd/Desktop/NAIST-PBL1/descriptors_selected.csv')
df_traning_data = df_traning.loc[:,"MolLogP":"EState_VSA9"].values
df_traning_data=StandardScaler().fit_transform(df_traning_data)
df_traning.head()
reducer = umap.UMAP(random_state=2021)
embedding = reducer.fit_transform(df_traning_data)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
df_traning = pd.read_csv('/home/wzabsd/Desktop/NAIST-PBL1/descriptors_selected.csv')
df_traning.head()
df_traning_data = df_traning.loc[:,"FractionCSP3":"VSA_EState3"].values
df_traning_data=StandardScaler().fit_transform(df_traning_data)
reducer = umap.UMAP(random_state=24)
embedding = reducer.fit_transform(df_traning_data)
embedding.shape
import seaborn as sns
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=[sns.color_palette()[x] for x in df_traning.Class.map({"Active":1, "Inactive":0})])
plt.gca().set_aspect('equal', 'datalim')
plt.title('UMAP projection of the descriptors dataset', fontsize=24)

In [ ]:
penguins = pd.read_csv("https://github.com/allisonhorst/palmerpenguins/raw/5b5891f01b52ae26ad8cb9755ec93672f49328a8/data/penguins_size.csv")
penguins = penguins.dropna()
penguins.species_short.value_counts()
reducer = umap.UMAP()
penguin_data = penguins[
    [
        "culmen_length_mm",
        "culmen_depth_mm",
        "flipper_length_mm",
        "body_mass_g",
    ]
].values
scaled_penguin_data = StandardScaler().fit_transform(penguin_data)
embedding = reducer.fit_transform(scaled_penguin_data)
penguins.head()
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=[sns.color_palette()[x] for x in penguins.species_short.map({"Adelie":0, "Chinstrap":1, "Gentoo":2})])
plt.gca().set_aspect('equal', 'datalim')
plt.title('UMAP projection of the Penguin dataset', fontsize=24)

In [5]:
import os
os.chdir('/home/wzabsd/Desktop/NAIST-PBL1/')

In [7]:
from DeepPurpose import oneliner
from DeepPurpose.dataset import *
from DeepPurpose import dataset

X_drug, X_drug_names = dataset.read_file_repurposing_library("/home/wzabsd/Desktop/NAIST-PBL1/data/test.tab")

oneliner.repurpose(*load_SARS_CoV2_Protease_3CL(), 
                   X_repurpose=X_drug,
                   drug_names=X_drug_names,
                   pretrained_dir='/home/wzabsd/Desktop/NAIST-PBL1/save_folder/pretrained_models/DeepPurpose_BindingDB/',
                   finetune_epochs=100,
                   finetune_batch_size=128
                   )

Loading customized repurposing dataset...
Checking if pretrained directory is valid...
Beginning to load the pretrained models...
Using pretrained model and making predictions...
repurposing...
Drug Target Interaction Prediction Mode...
in total: 9465 drug-target pairs
encoding drug...
unique drugs: 9465


In [ ]:
import pandas as pd

df = pd.read_csv('/home/wzabsd/Desktop/NAIST-PBL1/datasets/3CL_enzymatic_activity-clean.tsv', sep='\t')
df.head()
df=df.iloc[:,[0,1]]
df.to_csv("/home/wzabsd/Desktop/data/test.tab",index=False,sep='\t')
